# Signal Subgraph Estimators

### This tutorial will introduce the following signal-subgraph estimators:
- Incoherent subgraph estimator
- Coherent subgraph estimator

Preliminaries

$n$ - the number of nodes in the graph
the number of samples of the graph model
the probability of a graph sample being of class 1

(Lil excerpt about what a signal-subgraph is)

The estimator that is used to find the signal-subgraph determines certain properties of the resulting subgraph.

the number of edges that must be present in the subgraph
the number of vertices that each edge in the subgraph must be incident to

Both estimators use (first param) to determine the size of the resulting subgraph. (second parameter) is only used for the coherent estimator, which constrains the subgraph to (second param) vertices. 

## Incoherent Signal-Subgraph Estimator

The incoherent estimator (...)

For this example we will randomly select 20 edges from a graph with 70 vertices. These edges will have distinct edge probabilities when conditioned upon different graph classes. 

In [ ]:
#randomly choose 20 from 70x70 array. Refer to local notebook used during sg.py development